# Convert Facenet Keras model to tflite

https://github.com/nyoki-mtl/keras-facenet

Using this version from github for david sandberg's tf facenet model that has been converted to keras.

Otherwise get the keras model from [here](https://drive.google.com/open?id=1aT4DW6k1RqoRmMzCkmysc_qq33leJecF)

In [14]:
from PIL import Image
import tensorflow as tf
import numpy as np
from scipy import spatial

### Upload the keras facenet.h5 model to the machine.

*get the keras model from [here](https://drive.google.com/open?id=1aT4DW6k1RqoRmMzCkmysc_qq33leJecF) if you didn't already download it from the steps above.*

In [16]:
from tensorflow.keras.models import load_model
model = load_model('facenet_keras.h5')


ValueError: bad marshal data (unknown type code)

# Convert

In [ ]:
converter = tf.contrib.lite.TFLiteConverter.from_keras_model_file("facenet_kera.h5")
tflite_model = converter.convert()
open("facenet_keras.tflite", "wb").write(tflite_model)


In [ ]:
!ls -l

In [ ]:
# download
files.download('facenet_keras.tflite')

# Test it out



In [ ]:
interpreter = tf.contrib.lite.Interpreter(model_path="facenet_keras.tflite")
interpreter.allocate_tensors()

In [ ]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_details

In [ ]:
# Test model on random input data.
input_shape = input_details[0]['shape']
# change the following line to feed into your own data.
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print(input_data.shape)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data.shape)
print(input_details)

### Function to extract the embedding of any given input image

In [ ]:
def extract_features(filename):
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]
  img = Image.open(filename)
  img = img.resize((width, height))

  input_data = np.expand_dims(img, axis=0)
  input_data = (np.float32(input_data) - 128) / 128

  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])
  results = np.squeeze(output_data)
  return results

### Sanity check
Do a sanity check to ensure everything is working fine. upload images that you want to compare to see if it makes sense

In [ ]:
feat1 =  extract_features("001.jpeg")
feat2 = extract_features("002.jpeg")
feat3 = extract_features("003.jpg")

dist1 = 1 - spatial.distance.cosine(feat1,feat2)
dist2 = 1 - spatial.distance.cosine(feat1,feat3)
print(dist1,dist2)